In [1]:
import pandas as pd
import numpy as np

In [2]:
API_key = 'B51wnmtnL30svioyDrNUrhl4h7We2HR8i-DQgKUNPsbVsiruZAItOeM6IHC8e5Y72G-d9TjHZFQHtKJjBrSCWCF0gLTvDlfr9u7bKce5jClTMT4ua-m0dtzxI6q1XXYx'

In [3]:
from yelpapi import YelpAPI
import argparse
from pprint import pprint
#see help code https://github.com/gfairchild/yelpapi/blob/master/examples/examples.py

In [4]:
import requests
import json
headers = {'Authorization': 'Bearer %s' % API_key}

In [5]:
yelp_api = YelpAPI(API_key)

In [6]:
data = pd.read_csv('data/chicago-food-inspections/food-inspections.csv')

In [7]:
data.head(10)

,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,...,Results,Violations,Latitude,Longitude,Location,Historical Wards 2003-2015,Zip Codes,Community Areas,Census Tracts,Wards
0,2320260,GOPUFF,GOPUFF,2684560.0,Grocery Store,Risk 3 (Low),1801 W WARNER AVE,CHICAGO,IL,60613.0,...,Fail,NaN,41.956846,-87.674395,"{'longitude': '41.956845683288854', 'latitude'...",NaN,NaN,NaN,NaN,NaN
1,2320248,THE REDHEAD PIANO BAR,THE REDHEAD PIANO BAR,2313942.0,Restaurant,Risk 3 (Low),16-18 W ONTARIO ST,CHICAGO,IL,60654.0,...,Pass w/ Conditions,NaN,41.893371,-87.628783,"{'longitude': '41.893370903547904', 'latitude'...",NaN,NaN,NaN,NaN,NaN
2,2320245,THE REDHEAD PIANO BAR,THE REDHEAD PIANO BAR,2313945.0,Restaurant,Risk 3 (Low),16-18 W ONTARIO ST,CHICAGO,IL,60654.0,...,Pass w/ Conditions,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.893371,-87.628783,"{'longitude': '41.893370903547904', 'latitude'...",NaN,NaN,NaN,NaN,NaN
3,2320269,LA CHAPARRITA,LA CHAPARRITA,2689521.0,Mobile Food Preparer,Risk 2 (Medium),2300 S THROOP ST,CHICAGO,IL,60608.0,...,Pass,"38. INSECTS, RODENTS, & ANIMALS NOT PRESENT - ...",41.850451,-87.658798,"{'longitude': '41.85045102427', 'latitude': '-...",NaN,NaN,NaN,NaN,NaN
4,2320232,WALGREENS #10771,WALGREENS #10771,1898193.0,Grocery Store,Risk 3 (Low),6460 W FULLERTON AVE,CHICAGO,IL,60707.0,...,Out of Business,NaN,41.923769,-87.787626,"{'longitude': '41.92376935343716', 'latitude':...",NaN,NaN,NaN,NaN,NaN
5,2320229,HIMALAYAN SHERPA KITCHEN,HIMALAYAN SHERPA KITCHEN,2670751.0,Restaurant,Risk 1 (High),2701 W LAWRENCE AVE,CHICAGO,IL,60625.0,...,Pass w/ Conditions,5. PROCEDURES FOR RESPONDING TO VOMITING AND D...,41.968442,-87.696304,"{'longitude': '41.968442053562335', 'latitude'...",NaN,NaN,NaN,NaN,NaN
6,2320247,7-ELEVEN #35750A,7-ELEVEN,2688817.0,Grocery Store,Risk 1 (High),1658 N MILWAUKEE AVE,CHICAGO,IL,60647.0,...,Pass,"38. INSECTS, RODENTS, & ANIMALS NOT PRESENT - ...",41.911443,-87.679334,"{'longitude': '41.911442870211495', 'latitude'...",NaN,NaN,NaN,NaN,NaN
7,2320255,7-ELEVEN #35750A,7-ELEVEN,2688816.0,Grocery Store,Risk 3 (Low),1658 N MILWAUKEE AVE,CHICAGO,IL,60647.0,...,Pass,NaN,41.911443,-87.679334,"{'longitude': '41.911442870211495', 'latitude'...",NaN,NaN,NaN,NaN,NaN
8,2320240,DOLLAR TREE #00645,DOLLAR TREE,2695077.0,Grocery Store,Risk 3 (Low),6460 W FULLERTON AVE,CHICAGO,IL,60707.0,...,Fail,5. PROCEDURES FOR RESPONDING TO VOMITING AND D...,41.923769,-87.787626,"{'longitude': '41.92376935343716', 'latitude':...",NaN,NaN,NaN,NaN,NaN
9,2320256,GOPUFF,GOPUFF,2684558.0,Grocery Store,Risk 3 (Low),1801 W WARNER AVE,CHICAGO,IL,60613.0,...,Fail,5. PROCEDURES FOR RESPONDING TO VOMITING AND D...,41.956846,-87.674395,"{'longitude': '41.956845683288854', 'latitude'...",NaN,NaN,NaN,NaN,NaN


In [12]:
start = 0 #Starting row for the scrapping
end = 1000 #Ending row for the scrapping

test_API = data[['DBA Name','Longitude','Latitude','Address','City','Zip','State']].iloc[start:end]
test_API['Yelp name']=np.NaN
test_API['Yelp review']=np.NaN
test_API['Yelp review count']=np.NaN
test_API['Yelp rating']=np.NaN
test_API.head()

,DBA Name,Longitude,Latitude,Address,City,Zip,State,Yelp name,Yelp review,Yelp review count,Yelp rating
0,GOPUFF,-87.674395,41.956846,1801 W WARNER AVE,CHICAGO,60613.0,IL,NaN,NaN,NaN,NaN
1,THE REDHEAD PIANO BAR,-87.628783,41.893371,16-18 W ONTARIO ST,CHICAGO,60654.0,IL,NaN,NaN,NaN,NaN
2,THE REDHEAD PIANO BAR,-87.628783,41.893371,16-18 W ONTARIO ST,CHICAGO,60654.0,IL,NaN,NaN,NaN,NaN
3,LA CHAPARRITA,-87.658798,41.850451,2300 S THROOP ST,CHICAGO,60608.0,IL,NaN,NaN,NaN,NaN
4,WALGREENS #10771,-87.787626,41.923769,6460 W FULLERTON AVE,CHICAGO,60707.0,IL,NaN,NaN,NaN,NaN


In [13]:
for i in range(len(test_API)):
    temp = test_API.iloc[i]
    response = yelp_api.business_match_query(name=temp['DBA Name'],
                                             address1=temp['Address'],
                                             city=temp['City'],
                                             state=temp['State'],
                                             country='US') #['businesses'][0]
    
    if len(response['businesses'])>0:
        test_API['Yelp name'].iloc[i] = response['businesses'][0]['name']
        id_ =  response['businesses'][0]['id']
        url = "https://api.yelp.com/v3/businesses/" + id_ + "/reviews"
        req = requests.get(url, headers=headers)
        #print('the status code is {}'.format(req.status_code))
        parsed = json.loads(req.text)
        rates = [parsed['reviews'][i]['rating'] for i in range(len(parsed['reviews']))]
        test_API['Yelp rating'].iloc[i] = np.mean(rates)
        test_API['Yelp review count'].iloc[i] = len(rates)
        test_API['Yelp review'].iloc[i] = str(rates)
    #print(temp['DBA Name'])
    #pprint(response)

In [14]:
test_API

,DBA Name,Longitude,Latitude,Address,City,Zip,State,Yelp name,Yelp review,Yelp review count,Yelp rating
0,GOPUFF,-87.674395,41.956846,1801 W WARNER AVE,CHICAGO,60613.0,IL,NaN,NaN,NaN,NaN
1,THE REDHEAD PIANO BAR,-87.628783,41.893371,16-18 W ONTARIO ST,CHICAGO,60654.0,IL,Redhead Piano Bar,"[5, 4, 5]",3.0,4.666667
2,THE REDHEAD PIANO BAR,-87.628783,41.893371,16-18 W ONTARIO ST,CHICAGO,60654.0,IL,Redhead Piano Bar,"[5, 4, 5]",3.0,4.666667
3,LA CHAPARRITA,-87.658798,41.850451,2300 S THROOP ST,CHICAGO,60608.0,IL,NaN,NaN,NaN,NaN
4,WALGREENS #10771,-87.787626,41.923769,6460 W FULLERTON AVE,CHICAGO,60707.0,IL,Walgreens,"[4, 3, 2]",3.0,3.000000
...,...,...,...,...,...,...,...,...,...,...,...
995,79TH BBQ,-87.642230,41.692267,11101 S HALSTED ST,CHICAGO,60628.0,IL,79th Street BBQ,"[5, 4, 1]",3.0,3.333333
996,Senn High School,-87.665262,41.989051,5900 N Glenwood AVE,CHICAGO,60660.0,IL,Nicholas Senn High School,"[4, 1, 5]",3.0,3.333333
997,OLD GROUNDS SOCIAL,-87.653338,41.929124,950-952 W WRIGHTWOOD AVE,CHICAGO,60614.0,IL,Old Grounds Social,"[5, 3, 2]",3.0,3.333333
998,MARIANO'S #8516,-87.668573,41.920245,2112 N ASHLAND AVE,CHICAGO,60614.0,IL,NaN,NaN,NaN,NaN


In [32]:
print("Among the rows", start,"and", end,"we scrapped",len(test_API['Yelp name'])-np.count_nonzero(pd.isnull(test_API['Yelp name'])),"restaurants.")

Among the rows 0 and 1000 we scrapped 647 restaurants.
